In [7]:
import json
import time
from datetime import datetime
from openai import OpenAI
import pandas as pd
from IPython.display import display, HTML
import sys
sys.path.append('..')
# 프롬프트 템플릿 로드
from prompts import SYSTEM_PROMPT_TEMPLATE


In [8]:
# 1. JSON 파일 로드
with open("/home/chlee/Projects/LLM-bench/priming_without_mention_of_mbti.json", "r", encoding="utf-8") as f:
    MBTI_PRIMING = json.load(f)

# 말투 가이드라인 로드
with open("/home/chlee/Projects/LLM-bench/mbti_guides.json", "r", encoding="utf-8") as f:
    MBTI_GUIDES = json.load(f)
    
# 테스트 케이스 로드
with open("/home/chlee/Projects/LLM-bench/test_cases.json", "r", encoding="utf-8") as f:
    TEST_CASES = json.load(f)

In [9]:
ENERGY_GUIDES = MBTI_GUIDES["energy"]
PERCEPTION_GUIDES = MBTI_GUIDES["perception"]
JUDGMENT_GUIDES = MBTI_GUIDES["judgment"]
LIFESTYLE_GUIDES = MBTI_GUIDES["lifestyle"]

In [10]:
print(f"✅ MBTI 유형 수: {len(MBTI_PRIMING)}")
print(f"✅ 테스트 케이스 수: {len(TEST_CASES)}")

✅ MBTI 유형 수: 16
✅ 테스트 케이스 수: 10


In [11]:
def get_level(score: int) -> str:
    """1-10 점수를 low/mid/high로 변환"""
    if score <= 3:
        return "low"
    elif score <= 6:
        return "mid"
    else:
        return "high"

In [12]:
def build_system_prompt(
    name: str,
    gender: str,
    position: str,
    grade: str,
    biography: str,
    mbti_description: str,
    energy: int,
    perception: int,
    judgment: int,
    lifestyle: int
) -> str:
    """
    캐릭터 정보와 MBTI를 기반으로 시스템 프롬프트 생성
    """
    # 각 축 점수에 따른 해당 범위 가이드만 선택
    energy_guide = ENERGY_GUIDES[get_level(energy)]
    perception_guide = PERCEPTION_GUIDES[get_level(perception)]
    judgment_guide = JUDGMENT_GUIDES[get_level(judgment)].format(name=name)
    lifestyle_guide = LIFESTYLE_GUIDES[get_level(lifestyle)]
    
    prompt = SYSTEM_PROMPT_TEMPLATE.format(
        name=name,
        gender=gender,
        position=position,
        grade=grade,
        biography=biography,
        mbti_description=mbti_description,
        energy_guide=energy_guide,
        perception_guide=perception_guide,
        judgment_guide=judgment_guide,
        lifestyle_guide=lifestyle_guide,
    )
    
    return prompt

In [13]:
"""
유저가 캐릭터 생성/팔로우하면:

서버에서 {"role": "user", "content": "(채팅방에 입장했습니다)"} 자동 주입
LLM이 첫 인사 메시지 생성
유저에게 표시
    - temperature 기본값 0.4: 일관성 있는 첫 인사를 위해 낮게 설정
"""
def get_greeting(client, model_id: str, system_prompt: str, max_tokens: int = 200, temperature=0.4):
    """시스템 프롬프트로 첫 인사 메시지 생성"""
    start_time = time.time()
    
    try:
        completion = client.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": "(채팅방에 입장했습니다)"}
            ],
            max_tokens=max_tokens,
            temperature=0.8,
        )
        
        elapsed_time = time.time() - start_time
        response_text = completion.choices[0].message.content
        usage = completion.usage
        
        return {
            "response": response_text.strip() if response_text else None,
            "prompt_tokens": usage.prompt_tokens if usage else 0,
            "completion_tokens": usage.completion_tokens if usage else 0,
            "total_tokens": usage.total_tokens if usage else 0,
            "elapsed_time": round(elapsed_time, 2),
            "error": None
        }
    
    except Exception as e:
        return {
            "response": None,
            "prompt_tokens": 0,
            "completion_tokens": 0,
            "total_tokens": 0,
            "elapsed_time": time.time() - start_time,
            "error": str(e)
        }

In [14]:
# 가격 출처: https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/
MODELS = {
    "gpt-4o-mini": "openai/gpt-4o-mini",
}

MODEL_PRICING = {
    "gpt-4o-mini": {"input": 0.15, "output": 0.60},  # per 1M tokens
}

# OpenRouter 클라이언트 설정
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

In [15]:
results = []

print("=" * 70)
print(f"🎤 아이돌 첫 인사 메시지 벤치마크")
print(f"   테스트 케이스: {len(TEST_CASES)}")
print("=" * 70)

for model_name, model_id in MODELS.items():
    print(f"\n🤖 모델: {model_name}")
    pricing = MODEL_PRICING.get(model_name, {"input": 1.0, "output": 5.0})
    
    for i, case in enumerate(TEST_CASES, 1):
        # MBTI description 가져오기
        mbti_description = MBTI_PRIMING.get(case["mbti"], "")
        
        # 시스템 프롬프트 생성
        system_prompt = build_system_prompt(
            name=case["name"],
            gender=case["gender"],
            position=case["position"],
            grade=case["grade"],
            biography=case["biography"],
            mbti_description=mbti_description,
            energy=case["energy"],
            perception=case["perception"],
            judgment=case["judgment"],
            lifestyle=case["lifestyle"],
        )
        
        # API 호출
        result = get_greeting(client, model_id, system_prompt)
        
        # 비용 계산
        input_cost = result["prompt_tokens"] / 1_000_000 * pricing["input"]
        output_cost = result["completion_tokens"] / 1_000_000 * pricing["output"]
        cost = input_cost + output_cost
        
        results.append({
            "model": model_name,
            "case_id": i,
            "name": case["name"],
            "mbti": case["mbti"],
            "energy": case["energy"],
            "perception": case["perception"],
            "judgment": case["judgment"],
            "lifestyle": case["lifestyle"],
            "position": case["position"],
            "grade": case["grade"],
            "biography": case["biography"],
            "response": result["response"],
            "prompt_tokens": result["prompt_tokens"],
            "completion_tokens": result["completion_tokens"],
            "total_tokens": result["total_tokens"],
            "elapsed_time": result["elapsed_time"],
            "cost_usd": round(cost, 6),
            "error": result["error"]
        })
        
        # 진행 상황 출력
        status = "✅" if result["response"] else "❌"
        e_lvl = get_level(case["energy"])[0].upper()
        p_lvl = get_level(case["perception"])[0].upper()
        j_lvl = get_level(case["judgment"])[0].upper()
        l_lvl = get_level(case["lifestyle"])[0].upper()
        
        print(f"  {status} [{case['mbti']}] {case['name']} ({case['position']}) [E:{e_lvl}/N:{p_lvl}/F:{j_lvl}/P:{l_lvl}]: {result['elapsed_time']}s")
        if result["response"]:
            preview = result["response"][:70] + "..." if len(result["response"]) > 70 else result["response"]
            print(f"     💬 {preview}")
        
        time.sleep(0.3)

# DataFrame 생성
df = pd.DataFrame(results)
print(f"\n✅ 벤치마크 완료! 총 {len(df)}개 결과")

🎤 아이돌 첫 인사 메시지 벤치마크
   테스트 케이스: 10

🤖 모델: gpt-4o-mini
  ✅ [ENFP] 하윤 (메인 보컬) [E:H/N:H/F:H/P:H]: 4.01s
     💬 헤이!! 드디어 만났다!! ✨ 너무 반가워서 심장이 두근두근 해! 우리 만남은 운명인 것 같아💖 어떤 이야기 나눌까? 혹시 나...
  ✅ [ESTJ] 민준 (리더) [E:H/N:L/F:L/P:L]: 2.42s
     💬 헤이!! 드디어 만났다!! 🎉 먼저 제 소개를 하자면, 나는 민준이고, 우리 그룹의 리더야! 오늘 연습 끝나고 바로 이곳에 왔...
  ✅ [ESFP] 서아 (메인 댄서) [E:H/N:L/F:H/P:H]: 1.67s
     💬 헤이!! 드디어 만났다!! 🎉 오늘 연습 끝나고 바로 여기로 왔어! 진짜 기다렸어 ㅠㅠ 너무 반가워!! 💖 아 맞다, 그보다 ...
  ✅ [INFP] 지호 (서브 보컬) [E:L/N:H/F:H/P:H]: 1.56s
     💬 음... 안녕... 처음이라 좀 긴장되네요. 우리 만남은 운명인 것 같아. 정말 기다렸어요 ㅠㅠ 너무 반가워!! 우리 함께 멋...
  ✅ [ENTJ] 예린 (메인 래퍼) [E:H/N:H/F:L/P:L]: 1.4s
     💬 헤이!! 드디어 만났다!! 🎉 나는 예린이야! 우리 그룹의 메인 래퍼로 활약하고 있어! 함께 멋진 추억 만들어보자! 잘 부탁해...
  ✅ [ISFP] 현우 (비주얼) [E:L/N:L/F:H/P:H]: 1.03s
     💬 음... 안녕! 처음이라 좀 긴장되네요. 오늘 연습 끝나고 바로 왔어요. 정말 기다렸어요 ㅠㅠ 너무 반가워!! 우리 어떤 이야...
  ✅ [ENTP] 소율 (막내) [E:H/N:H/F:L/P:H]: 0.94s
     💬 헤이!! 드디어 만났다!! 🎉✨ 난 소율이야, 우리 그룹의 귀염둥이 막내! 😄 오늘은 어떤 신나는 이야기를 나눌까? 우리 같이...
  ✅ [ISTJ] 태민 (서브 댄서) [E:L/N:L/F:L/P:L]: 2.73s
     💬 음... 

### 결과 예쁘게

In [18]:
# 결과 카드 뷰
cards_html = ""

for _, row in df.iterrows():
    e_level = get_level(row['energy'])
    p_level = get_level(row['perception'])
    j_level = get_level(row['judgment'])
    l_level = get_level(row['lifestyle'])
    
    level_colors = {"high": "#2e7d32", "mid": "#f9a825", "low": "#c2185b"}
    
    cards_html += f"""
    <div style="background: white; border-radius: 15px; padding: 20px; margin-bottom: 15px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
        <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
            <span style="font-size: 1.2em; font-weight: bold;">👤 {row['name']}</span>
            <div>
                <span style="background: linear-gradient(135deg, #667eea, #764ba2); color: white; padding: 3px 10px; border-radius: 15px; font-size: 0.85em; margin-right: 5px;">{row['mbti']}</span>
                <span style="background: #e8f4fd; color: #1976d2; padding: 3px 10px; border-radius: 15px; font-size: 0.85em; margin-right: 5px;">{row['position']}</span>
                <span style="background: #fff3e0; color: #f57c00; padding: 3px 10px; border-radius: 15px; font-size: 0.85em;">등급 {row['grade']}</span>
            </div>
        </div>
        <div style="display: flex; gap: 15px; margin-bottom: 10px; font-size: 0.9em;">
            <span>에너지: <b>{row['energy']}</b> <span style="color: {level_colors[e_level]};">[{e_level.upper()}]</span></span>
            <span>인식: <b>{row['perception']}</b> <span style="color: {level_colors[p_level]};">[{p_level.upper()}]</span></span>
            <span>판단: <b>{row['judgment']}</b> <span style="color: {level_colors[j_level]};">[{j_level.upper()}]</span></span>
            <span>생활: <b>{row['lifestyle']}</b> <span style="color: {level_colors[l_level]};">[{l_level.upper()}]</span></span>
        </div>
        <div style="background: #f8f9fa; padding: 10px; border-radius: 8px; font-style: italic; color: #666; margin-bottom: 10px; font-size: 0.9em;">📝 {row['biography']}</div>
        <div style="background: linear-gradient(135deg, #f8f9fa, #e9ecef); padding: 15px; border-radius: 10px; border-left: 4px solid #667eea;">💬 {row['response'] or '(응답 없음)'}</div>
        <div style="display: flex; gap: 20px; margin-top: 10px; font-size: 0.85em; color: #888;">
            <span>⏱️ {row['elapsed_time']}초</span>
            <span>🔢 {row['total_tokens']} 토큰</span>
            <span>💵 ${row['cost_usd']:.6f}</span>
        </div>
    </div>
    """

display(HTML(cards_html))

In [19]:
def generate_html_report(df: pd.DataFrame, filename: str):
    """HTML 리포트 생성"""
    avg_cost = df["cost_usd"].mean()
    avg_tokens = df["total_tokens"].mean()
    avg_time = df["elapsed_time"].mean()
    
    html_content = f"""<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>아이돌 첫 인사 벤치마크 결과</title>
    <style>
        body {{ font-family: -apple-system, BlinkMacSystemFont, sans-serif; background: linear-gradient(135deg, #667eea, #764ba2); min-height: 100vh; padding: 20px; margin: 0; }}
        .container {{ max-width: 1000px; margin: 0 auto; }}
        h1 {{ color: white; text-align: center; }}
        .summary {{ display: flex; gap: 20px; justify-content: center; margin-bottom: 30px; flex-wrap: wrap; }}
        .summary-card {{ background: white; border-radius: 15px; padding: 20px; text-align: center; min-width: 120px; }}
        .summary-card .value {{ font-size: 1.5em; font-weight: bold; color: #667eea; }}
        .card {{ background: white; border-radius: 15px; padding: 20px; margin-bottom: 15px; }}
        .response {{ background: #f8f9fa; padding: 15px; border-radius: 10px; border-left: 4px solid #667eea; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>🎤 아이돌 첫 인사 벤치마크 결과</h1>
        <div class="summary">
            <div class="summary-card"><div class="value">{len(df)}</div><div>테스트 케이스</div></div>
            <div class="summary-card"><div class="value">{avg_tokens:.0f}</div><div>평균 토큰</div></div>
            <div class="summary-card"><div class="value">${avg_cost:.6f}</div><div>평균 비용</div></div>
            <div class="summary-card"><div class="value">{avg_time:.2f}s</div><div>평균 응답시간</div></div>
        </div>
"""
    
    for _, row in df.iterrows():
        html_content += f"""
        <div class="card">
            <div><b>👤 {row['name']}</b> | {row['mbti']} | {row['position']} | 등급 {row['grade']}</div>
            <div style="margin: 10px 0; color: #666;">📝 {row['biography']}</div>
            <div class="response">💬 {row['response'] or '(응답 없음)'}</div>
            <div style="margin-top: 10px; color: #888;">⏱️ {row['elapsed_time']}s | 🔢 {row['total_tokens']} 토큰 | 💵 ${row['cost_usd']:.6f}</div>
        </div>
"""
    
    html_content += "</div></body></html>"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html_content)
    
    print(f"📊 HTML 저장: {filename}")

# HTML 리포트 생성
html_filename = f"benchmark_results_{timestamp}.html"
generate_html_report(df, html_filename)

📊 HTML 저장: benchmark_results_20260205_164248.html
